In [1]:


# import sys
# !{sys.executable} -m pip install --upgrade git+https://github.com/huggingface/transformers
# !{sys.executable} -m pip install --upgrade torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# !{sys.executable} -m pip install --upgrade onnxruntime==1.4.0
# !{sys.executable} -m pip install -i https://test.pypi.org/simple/ ort-nightly
# !{sys.executable} -m pip install --upgrade onnxruntime-tools

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# from transformers import BertForTokenClassification, BertTokenizerFast
# import torch


# my_model_name = "DeepPavlov/rubert-base-cased" # replace here by the name of your model
# tokenizer = BertTokenizerFast.from_pretrained(my_model_name )
# model = BertModel.from_pretrained(my_model_name )
# size = 254 # the max length you expect for your model. Don't forget the two extra tokens of start and end, here 510 + 2 to make 512 which is the max length allowed for all the models (except longformer).

# inputs_dict = tokenizer("hello" * [size])
# model._saved_model_inputs_spec = None
# model._set_save_spec(inputs_dict)
# model.save_pretrained("../out/token_clf")

In [5]:
# !rm -rf onnx/
# from pathlib import Path
# from transformers.convert_graph_to_onnx import convert

# # Handles all the above steps for you
# convert(framework="pt", model="../out/token_clf", output=Path("onnx/bert-base-cased.onnx"), opset=11)

In [6]:
%cd /Users/ibragim/Documents/punctuation-restoration/src

/Users/ibragim/Documents/punctuation-restoration/src


In [22]:
from model import DeepPunctuation
import torch

deep_punctuation = DeepPunctuation('DeepPavlov/rubert-base-cased')

In [5]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased')

In [14]:
train_dataset = tokenizer(['привет'],  return_tensors='pt')

In [20]:
dummy_input[0]

tensor([[[  101, 26856,   102]]])

In [47]:
from torch.onnx import export

device = torch.device("cpu") 

model_onnx_path = "model.onnx"
dummy_input = (
    train_dataset["input_ids"].to(device), 
    train_dataset["attention_mask"].to(device)
)
input_names = ["input_ids", "attention_mask"]
output_names = ["logits"]
export(
    deep_punctuation, dummy_input, model_onnx_path, input_names = input_names, 
    output_names = output_names, verbose=False, opset_version=11
)

/usr/local/anaconda3/envs/py37/lib/python3.7/site-packages/transformers/models/bert/modeling_bert.py:194: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]
/usr/local/anaconda3/envs/py37/lib/python3.7/site-packages/transformers/modeling_utils.py:1768: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape[chunk_dim] == tensor_shape for input_tensor in input_tensors


RuntimeError: required keyword attribute 'value' has the wrong type

In [25]:
import torch.quantization
quantized_model = torch.quantization.quantize_dynamic(deep_punctuation, {torch.nn.Linear}, dtype=torch.qint8)

In [46]:
# set quantization config for server (x86)
deploymentmyModel.qconfig = torch.quantization.get_default_config('fbgemm')

# insert observers
quantized_model_2 = torch.quantization.prepare(deep_punctuation, inplace=False)
# Calibrate the model and collect statistics

# convert to quantized version
quantized_model_2 = torch.quantization.convert(deep_punctuation, inplace=False)

AttributeError: module 'torch.quantization' has no attribute 'get_default_config'

In [27]:
import os

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(deep_punctuation)
print_size_of_model(quantized_model)

Size (MB): 749.328227
Size (MB): 492.797163


In [34]:

def evaluate(model_, data_source):
# Turn on evaluation mode which disables dropout.
    model_.eval()
    total_loss = 0.
    with torch.no_grad():
        tr = model_(data_source['input_ids'], data_source['attention_mask'])
    return tr

In [45]:
td = ' '.join(['привет']*120)
test_data = tokenizer([td]*12,  return_tensors='pt')

import time
def time_model_evaluation(model, test_data):
    s = time.time()
    loss = evaluate(model, test_data)
    elapsed = time.time() - s
    print(f'''elapsed time {elapsed}''')

time_model_evaluation(deep_punctuation, test_data)
time_model_evaluation(quantized_model, test_data)

elapsed time 1.9286987781524658
elapsed time 2.84853458404541


In [9]:
!python3 -m transformers convert_graph_to_onnx.py --framework pt --model mrm8488/mobilebert-finetuned-ner bert-base-cased.onnx

/usr/local/anaconda3/envs/py37/bin/python3: No module named transformers.__main__; 'transformers' is a package and cannot be directly executed


In [18]:
from transformers.convert_graph_to_onnx import convert
from pathlib import Path
pth = Path("onnx/bert-test-cased.onnx")
# Handles all the above steps for you
convert(framework="pt", model="mrm8488/mobilebert-finetuned-ner", output=pth, opset=11)
# ...


ONNX opset version set to: 11
Loading pipeline (model: mrm8488/mobilebert-finetuned-ner, tokenizer: mrm8488/mobilebert-finetuned-ner)
Using framework PyTorch: 1.6.0
Found input input_ids with shape: {0: 'batch', 1: 'sequence'}
Found input token_type_ids with shape: {0: 'batch', 1: 'sequence'}
Found input attention_mask with shape: {0: 'batch', 1: 'sequence'}
Found output output_0 with shape: {0: 'batch', 1: 'sequence'}
Found output output_1 with shape: {0: 'batch'}
Ensuring inputs are in correct order
position_ids is not present in the generated input list.
Generated inputs order: ['input_ids', 'attention_mask', 'token_type_ids']


RuntimeError: required keyword attribute 'value' has the wrong type

In [5]:
!pwd

/Users/ibragim


In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-base-finetuned-sentiment")
cpu_model = create_model_for_provider("onnx/bert-test-cased.onnx", "CPUExecutionProvider")

# Inputs are provided through numpy array
model_inputs = tokenizer.encode_plus("My name is Bert", return_tensors="pt")
inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}

# Run the model (None = get all the outputs)
sequence, pooled = cpu_model.run(None, inputs_onnx)

# Print information about outputs

print(f"Sequence output: {sequence.shape}, Pooled output: {pooled.shape}")

pytorch_model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-base-finetuned-sentiment")
a, = pytorch_model(**model_inputs)
print(f"finetune non onnx pytorch model output: {a.shape}")
# ...

In [27]:
r"([\p{S}\p{P}]+)"

'([\\p{S}\\p{P}]+)'

In [30]:
import regex

REGEX0 = regex.compile(r"([\p{S}\p{P}]+)")
REGEX1 = regex.compile(r" +")


In [31]:

text = 'Привет. как твои дела? Чего делаешь, а?'

tokens = (
        REGEX1.sub(" ", REGEX0.sub(r" \1 ", text.lower()))
        .strip()
        .split(" ")
    )

In [32]:
tokens

['привет', '.', 'как', 'твои', 'дела', '?', 'чего', 'делаешь', ',', 'а', '?']